In [1]:
import pandas as pd
import json 
import gzip
import os

import nltk
from nltk.corpus import stopwords
import string
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [2]:
data_path = '../src/data'
mapping_corpus_path_2 = data_path + r'/product/lspc2020_to_tablecorpus/Cleaned'
notebook_path = '../notebooks'
product_path = os.path.join(data_path, 'product')

In [ ]:
#cluster_df = pd.read_json(os.path.join(mapping_corpus_path_2, 'df_large_matched.json'), compression='gzip', orient='records', lines=True)
clothes_cluster_df = pd.read_json(os.path.join(mapping_corpus_path_2, 'joined_clothes_v3.json'), compression='gzip', orient='records', lines=True)
#bikes_cluster_df = pd.read_json(os.path.join(mapping_corpus_path_2, 'joined_bikes.json'), compression='gzip', orient='records', lines=True)
#cars_cluster_df = pd.read_json(os.path.join(mapping_corpus_path_2, 'joined_cars.json'), compression='gzip', orient='records', lines=True)
#drugstore_cluster_df = pd.read_json(os.path.join(mapping_corpus_path_2, 'joined_drugstore.json'), compression='gzip', orient='records', lines=True)
#electronics_cluster_df = pd.read_json(os.path.join(mapping_corpus_path_2, 'joined_electronics_v3.json'), compression='gzip', orient='records', lines=True)
#technology_cluster_df = pd.read_json(os.path.join(mapping_corpus_path_2, 'joined_technology.json'), compression='gzip', orient='records', lines=True)
#tools_cluster_df = pd.read_json(os.path.join(mapping_corpus_path_2, 'joined_tools.json'), compression='gzip', orient='records', lines=True)
#random_cluster_df = pd.read_json(os.path.join(mapping_corpus_path_2, 'joined_random.json'), compression='gzip', orient='records', lines=True)

In [ ]:
clothes_final_entities_df = pd.read_csv(os.path.join(mapping_corpus_path_2, 'Clothes_cluster_8_tables.csv'), index_col=None)
clothes_final_entities_list = clothes_final_entities_df['cluster_id']

In [ ]:
clothes_clusters_all_8_df = clothes_cluster_df[clothes_cluster_df['cluster_id'].isin(clothes_final_entities_list)]
#bikes_clusters_all_8_df = bikes_cluster_df[bikes_cluster_df['cluster_id'].isin(bikes_final_entities_list)]
#cars_clusters_all_8_df = cars_cluster_df[cars_cluster_df['cluster_id'].isin(cars_final_entities_list)]
#drugstore_clusters_all_8_df = drugstore_cluster_df[drugstore_cluster_df['cluster_id'].isin(drugstore_final_entities_list)]
#electronics_clusters_all_8_df = electronics_cluster_df[electronics_cluster_df['cluster_id'].isin(electronics_final_entities_list)]
#technology_clusters_all_8_df = technology_cluster_df[technology_cluster_df['cluster_id'].isin(technology_final_entities_list)]
#tools_clusters_all_8_df = tools_cluster_df[tools_cluster_df['cluster_id'].isin(tools_final_entities_list)]
#random_clusters_all_8_df = random_cluster_df[random_cluster_df['cluster_id'].isin(random_final_entities_list)]

clothes_clusters_all_8_df.to_csv(os.path.join(mapping_corpus_path_2, 'Clothes_clusters_all_8_tables_v3.csv'), columns=None)
#bikes_clusters_all_8_df.to_csv(os.path.join(mapping_corpus_path_2, 'Bikes_clusters_all_8_tables.csv'), columns=None)
#cars_clusters_all_8_df.to_csv(os.path.join(mapping_corpus_path_2, 'Cars_clusters_all_8_tables.csv'), columns=None)
#drugstore_clusters_all_8_df.to_csv(os.path.join(mapping_corpus_path_2, 'Drugstore_clusters_all_8_tables.csv'), columns=None)
#electronics_clusters_all_8_df.to_csv(os.path.join(mapping_corpus_path_2, 'Electronics_clusters_all_8_tables.csv'), columns=None)
#technology_clusters_all_8_df.to_csv(os.path.join(mapping_corpus_path_2, 'Technology_clusters_all_8_tables.csv'), columns=None)
#tools_clusters_all_8_df.to_csv(os.path.join(mapping_corpus_path_2, 'Tools_clusters_all_8_tables.csv'), columns=None)
#random_clusters_all_8_df.to_csv(os.path.join(mapping_corpus_path_2, 'Random_clusters_all_8_tables.csv'), columns=None)